<a href="https://colab.research.google.com/github/Nallasailaja/FMML-/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits typically provides more consistent and reliable results. Here's why:


---

Benefits of Averaging Across Multiple Splits

1. Reduces Variance:

A single split of the data can produce biased or unstable results due to random variation in how the data is divided into training and validation sets.

Averaging over multiple splits (e.g., in k-fold cross-validation) smooths out these fluctuations.



2. Better Generalization Estimate:

By evaluating performance across different subsets, you get a more accurate representation of how the model generalizes to unseen data.



3. Mitigates Overfitting to Specific Splits:

A single split might have an unusual distribution or class imbalance, skewing the accuracy.

Multiple splits ensure that each part of the dataset contributes to both training and validation, balancing the evaluation.



4. Handles Data Variability:

When your dataset is small or imbalanced, multiple splits help account for variability in the data.





---

Common Methods

1. k-Fold Cross-Validation:
Divide the data into k folds and rotate the validation set across all folds.

The final accuracy is the mean of accuracies from all folds.

Example: 5-fold cross-validation averages results across 5 splits.



2. Repeated k-Fold Cross-Validation:
Repeat the k-fold process multiple times with different splits to further stabilize the results.


3. Stratified Splits:
For imbalanced data, use stratified splits to maintain class distribution across splits.




---

Caveats

Computational Cost: Multiple splits increase computation time, as the model is trained and validated multiple times.

Potential Data Leakage: Ensure data preprocessing (e.g., scaling, encoding) is done within each fold to avoid leakage between training and validation.



---

When Not to Average

If the splits are not representative of the data (e.g., random splits without stratification in imbalanced datasets), the average may not be meaningful.

Always check standard deviation along with the mean to understand the stability of your result

2. Does it give more accurate estimate of test accuracy?
Yes, averaging validation accuracy across multiple splits (e.g., using k-fold cross-validation) generally provides a more accurate estimate of test accuracy, but with some caveats. Here's why:


---

Why It Improves Test Accuracy Estimation

1. Reduced Overfitting to a Single Split

A single validation set might not represent the overall data distribution, leading to an over- or under-estimation of test accuracy.

Multiple splits average out this bias, giving a better estimate of the model's performance on unseen data.



2. Better Utilization of Data

In k-fold cross-validation, every data point is used for validation exactly once and for training  times.

This ensures the model learns from and is tested on all parts of the dataset, making the evaluation more robust.



3. Accounts for Data Variability

If certain splits of the data are easier or harder to predict, averaging across multiple splits provides a more balanced accuracy estimate.



4. Handles Imbalanced Datasets

Techniques like stratified k-fold cross-validation maintain class proportions across splits, improving accuracy estimation for imbalanced datasets.





---

Comparison to Single-Split Validation

Single-Split Validation: Test accuracy estimate can vary greatly depending on how the data is split (e.g., 80% train, 20% validation). This split might not represent the true distribution of the test data.

Cross-Validation: Provides a more reliable and less noisy estimate by testing on multiple subsets and averaging the results.



---

Caveats

1. Still an Estimate:
Cross-validation provides an estimate of test accuracy, not the exact test accuracy. Differences can arise due to:

Test data having a slightly different distribution.

Noise or randomness in the data.



2. Data Leakage Risk:
If preprocessing (e.g., normalization, feature selection) is done incorrectly (e.g., before splitting), it may lead to overly optimistic results.


3. Standard Deviation Matters:
Even with averaging, a high standard deviation across splits indicates the model may perform inconsistently on test data.




---

When It Doesn't Help

If the training and test data come from different distributions, cross-validation won't accurately estimate test accuracy. Instead, domain-specific validation methods or additional holdout test sets may be needed.



---

Conclusion

Averaging validation accuracy across multiple splits typically gives a more reliable and accurate estimate of test accuracy, especially when the data is small, imbalanced, or noisy. However, it's not a guarantee, and care must be taken to avoid pitfalls like data leakage.

Would you like a demonstration of this in practice?


3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The number of iterations in a cross-validation process (e.g., k-fold cross-validation or repeated k-fold cross-validation) affects the reliability and stability of the test accuracy estimate. Here's the impact of increasing the number of iterations:


---

Effect of the Number of Iterations

1. Improved Stability:

With more iterations (e.g., more folds or repeated k-fold), the estimate becomes more consistent by reducing the influence of random variations in the data splits.

The standard deviation of accuracy across folds decreases, making the estimate more reliable.



2. Diminishing Returns:

After a certain point, increasing the number of iterations provides minimal benefit. This is because the accuracy estimate converges toward the true test accuracy as you average over more splits.



3. Robustness to Outliers:

Higher iterations reduce the effect of any single "unlucky" split that might have been biased (e.g., containing harder-to-predict samples in the validation set).



4. Better Utilization of Data:

In repeated k-fold, every data point participates in validation multiple times across different splits, leading to a broader evaluation of performance.





---

When Higher Iterations May Not Help

1. Computational Cost:

More iterations increase training time since the model is trained and validated multiple times.

For complex models, this can become infeasible.



2. Noisy Data:

If the dataset is noisy or very small, increasing iterations may not significantly improve the estimate, as the underlying data variability limits the accuracy.



3. Convergence Limit:

After sufficient iterations (e.g., 10-20 folds or repetitions), the accuracy estimate often stabilizes. Further iterations may only yield marginal improvements.





---

Recommendation

Small Datasets: Use a higher number of folds (e.g., 10 or more) or repeated k-fold cross-validation to ensure robustness.

Larger Datasets: Fewer iterations (e.g., 5-10 folds) are often sufficient since the law of large numbers ensures a stable estimate.



---

Conclusion

Higher iterations generally improve the accuracy estimate's reliability but have diminishing returns beyond a certain point. The optimal number depends on your dataset size, variability, and computational resources. Would you like guidance on selecting the number of iterations for your specific use case?



4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations (or folds) in cross-validation can help mitigate the issue of a very small training or validation dataset to some extent, but it doesn't entirely solve the problem. Here's why:


---

Effect of Small Train/Validation Dataset

1. Limited Training Data:

If the training dataset is very small, the model may not be able to learn effectively, even with increased iterations. Each model trained on such a small dataset will be heavily influenced by noise, potentially leading to overfitting.

Increasing iterations might allow the model to train on more subsets of the data, but the quality of the model will still be limited by the small amount of data available.



2. Limited Validation Data:

For small validation datasets, cross-validation helps by using different portions of the data for validation, but it still doesn't compensate for the lack of a large, representative validation set. If the validation set is too small, it might not fully capture the model’s performance on unseen data.





---

What Increasing Iterations Helps With

1. Better Utilization of Available Data:

In k-fold cross-validation, each data point gets used for training  times and for validation once, meaning every data point contributes to both training and validation. This helps make the most out of the limited data.

Repeated k-fold cross-validation allows the model to be evaluated on different random splits, further reducing the impact of any single data split.



2. Reducing Bias from One Split:

Increasing the number of folds or repetitions can smooth out any irregularities or biases from a single split, especially in small datasets. This ensures that the performance estimate is more robust.





---

Limitations of Increasing Iterations with Small Datasets

1. Overfitting to Small Data:

While you might get a better estimate of performance, the model's ability to generalize may still be poor if the training data is too small. Cross-validation doesn't magically increase the amount of useful information available for training.



2. Bias in Evaluation:

A very small validation set may still fail to give an accurate estimate of test performance, even with more iterations. If the data is limited, there's a chance that the splits won't represent the entire population well.



3. Noise and Variability:

Small datasets are prone to high variability. Multiple iterations can help smooth out the estimate, but the inherent noise in the data will still affect the results.





---

Alternative Approaches to Handle Small Datasets

1. Data Augmentation:

For certain types of data (e.g., images, text), you can apply augmentation techniques (like rotation, scaling, cropping) to artificially increase the size of the training dataset.



2. Transfer Learning:

If possible, use a pre-trained model on a larger dataset and fine-tune it on your small dataset. This leverages knowledge learned from larger datasets.



3. Regularization:

Apply regularization techniques (like L2 regularization) to avoid overfitting on small datasets and to encourage the model to generalize better.



4. Synthetic Data Generation:

In some cases, generating synthetic data (e.g., using SMOTE for imbalanced data or other generative models) can help improve the model's ability to generalize.



5. Holdout Test Set:

If the dataset is too small for cross-validation, consider using a train-validation-test split. Ensure that the test set remains unseen during both training and validation phases to get an unbiased estimate of performance.





---

Conclusion

While increasing iterations in cross-validation can help by better utilizing small datasets and providing a more stable estimate, it does not fully address the fundamental problem of having too little data. If you are working with a very small dataset, other strategies like data augmentation, transfer learning, and regularization should be explored to improve model performance.

Would you like more details on any of these strategies?

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.